In [1]:
%load_ext autoreload 
%autoreload 2

import time
import torch
import os
import nltk
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import sys

sys.path.insert(0, '/home/bettyld/PJ/Documents/NLP_PJ/nlp_recommend')
%matplotlib inline


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from nlp_recommend import LoadData
corpus = LoadData(n_max=5000, random=False)
philo_df = corpus.corpus
philo_df = philo_df[['sentence', 'author', 'tok_lem_sentence']]
philo_df = philo_df.loc[(philo_df.tok_lem_sentence.str.len()<15)]
print(len(philo_df), device)
philo_df.head()


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/bettyld/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/bettyld/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/bettyld/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/home/bettyld/PJ/Documents/NLP_PJ/env/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


Removing numbers...
Removing proper noun... 4950


Processing: 100%|█████████████████████████████████| 9/9 [00:13<00:00,  1.45s/it]


Cleaning sentences...
Index(['index', 'sentence', 'author', 'title', 'clean_sentence',
       'tok_lem_sentence'],
      dtype='object')
828 cuda


,sentence,author,tok_lem_sentence
0,in the case of cognition according to concepts...,Kant,"[case, cognition, according, concept, believe,..."
1,Here then I end my whole critical undertaking.,Kant,"[end, whole, critical, undertaking]"
2,natural concepts and the concept of freedom.,Kant,"[natural, concept, concept, freedom]"
3,that cause which acts in accordance with conce...,Kant,"[cause, act, accordance, concept]"
4,But the last distinction is essential.,Kant,"[last, distinction, essential]"


# BERT Tokenizer
https://www.kaggle.com/eriknovak/pytorch-bert-sentence-similarity

In [3]:
# V6 Torch bert transformers
# import torch
# from transformers import BertModel, BertTokenizer

# bert_version = 'sentence-transformers/paraphrase-MiniLM-L6-v2'
# tokenizer = BertTokenizer.from_pretrained(bert_version)
# model = BertModel.from_pretrained(bert_version)



# Retrieving last hdden layer

## Tokenizer 
Return the ids of the tokens. You can overwrite argument per default
```
pt_batch = tokenizer(
    ["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."],
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt"
)
```

In [3]:
from transformers import AutoTokenizer, AutoModel, pipeline
import torch
#nitialize our model and tokenizer:
model_name = 'sentence-transformers/bert-base-nli-mean-tokens' # 

model_name = 'sentence-transformers/paraphrase-MiniLM-L6-v2' #best trade off

model_name = 'sentence-transformers/paraphrase-mpnet-base-v2'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

device = torch.cuda.device_count() - 1
device = -1
print('device', device)
classifier = pipeline('feature-extraction', model=model, tokenizer=tokenizer, device=device)

CORPUS = list(philo_df.sentence.values)


device -1


In [8]:
# Test tokenizer

sentences = ['This is an example of dish', 'Each sentence is converted to tokens']

print(tokenizer.tokenize(sentences[0]))

['this', 'is', 'an', 'example', 'of', 'dish']


## 1 - Hugging Face without pipeline

In [9]:
## REQUIRE SMALL BATCH SIZE or run on cpu
# tokenize sentence
device = 'cpu'
encoded_input = tokenizer(CORPUS[:10], padding=True, truncation=True, return_tensors='pt')
encoded_input = encoded_input.to(device)
model = model.to(device)
with torch.no_grad():
    model_output = model(**encoded_input)
# first element contains all tokens, secund alement??
print(model_output[0].shape, model_output[1].shape, len(model_output)) # 2,7, 384

torch.Size([10, 30, 768]) torch.Size([10, 768]) 2


In [10]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
sentence_embeddings.shape # shape nb_sentence, embedding shape (=384)

torch.Size([10, 768])

## 2 - Hugging face with pipeline

In [47]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device_no

0

In [6]:
# %%timeit

from tqdm import tqdm
# initialize dictionary: stores tokenized sentences

#CPU
device_no = -1
classifier = pipeline('feature-extraction', model=model, tokenizer=tokenizer, device=device_no)
device = 'cpu'

# GPU
# device_no = device = 0
# classifier = pipeline('feature-extraction', model=model, tokenizer=tokenizer, device=device)
# device = 'cuda'


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(token_embeddings, attention_mask):
    """torch"""
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def generate_embeddings(corpus, tokenizer, classifier, batch_size=8, small_memory=True):
    small_device = 'cpu' if small_memory else device
    nb_batchs = 1 if (len(corpus) < batch_size) else len(corpus) // batch_size
    mean_pooled = []
    batchs = np.array_split(corpus, nb_batchs)
    for idx, batch in tqdm(enumerate(batchs), total=nb_batchs): 
        batch = list(batch)
        embeds = classifier(batch)
        token_embeddings = torch.tensor(embeds).to(device)
        token_dict = tokenizer(
                        batch,
                        padding=True,
                        truncation=True,
                    #     max_length=512,
                        return_tensors="pt") 
        
        attention_mask =  token_dict['attention_mask'].to(device)
        embeds = mean_pooling(token_embeddings, attention_mask)
        if small_memory:
            embeds = embeds.to(small_device)
        mean_pooled.append(embeds)
    mean_pooled_tensor = torch.tensor(len(corpus), dtype=float).to(small_device)
    mean_pooled = torch.cat(mean_pooled, out=mean_pooled_tensor)
    return mean_pooled


pipeline_embeds = generate_embeddings(CORPUS, tokenizer, classifier)
pipeline_embeds.shape

100%|█████████████████████████████████████████| 103/103 [00:38<00:00,  2.70it/s]


torch.Size([828, 768])

## 3 - With sentence-transormer

In [23]:
from sentence_transformers import SentenceTransformer, util
import torch

sbert_model = SentenceTransformer(model_name)
#Compute embeddings
sbert_embeddings = sbert_model.encode(CORPUS, convert_to_tensor=True, device='cpu', batch_size=8)


KeyboardInterrupt: 

In [ ]:
sbert_embeddings.shape

## Comparison of three models
Embedding from three models equal at 10e-6 but very similars
Top_k results are the same between pipeline and sbert.

In [ ]:
# Embedding values
np.equal(sentence_embeddings.cpu().numpy(), pipeline_embeds)
sentence_embeddings.cpu().numpy()[0][:10], pipeline_embeds[0][:10]

In [ ]:
sbert_embeddings.cpu().numpy()[0][:10]

### Test inference time for sbert and pipelinebert and result equality

In [17]:
len(CORPUS)
test_sentence = 'Beautiful life'


In [15]:
%%timeit

# Best match pipeline
pipeline_out = generate_embeddings(test_sentence, tokenizer, classifier)[0]
cos_scores = util.pytorch_cos_sim(pipeline_out, pipeline_embeds)[0]
print(cos_scores.shape, sbert_embeddings.shape)
score, idx = torch.topk(cos_scores, k=5)
print('index', score, idx)
print(CORPUS[idx[0]])

(1, 4, 768)
torch.Size([828]) torch.Size([828, 384])
index tensor([0.6845, 0.6466, 0.6318, 0.6130, 0.5709], dtype=torch.float64) tensor([ 46, 468,  97, 105,  34])
The object of such satisfaction is called beautiful.
(1, 4, 768)
torch.Size([828]) torch.Size([828, 384])
index tensor([0.6845, 0.6466, 0.6318, 0.6130, 0.5709], dtype=torch.float64) tensor([ 46, 468,  97, 105,  34])
The object of such satisfaction is called beautiful.
(1, 4, 768)
torch.Size([828]) torch.Size([828, 384])
index tensor([0.6845, 0.6466, 0.6318, 0.6130, 0.5709], dtype=torch.float64) tensor([ 46, 468,  97, 105,  34])
The object of such satisfaction is called beautiful.
(1, 4, 768)
torch.Size([828]) torch.Size([828, 384])
index tensor([0.6845, 0.6466, 0.6318, 0.6130, 0.5709], dtype=torch.float64) tensor([ 46, 468,  97, 105,  34])
The object of such satisfaction is called beautiful.
(1, 4, 768)
torch.Size([828]) torch.Size([828, 384])
index tensor([0.6845, 0.6466, 0.6318, 0.6130, 0.5709], dtype=torch.float64) tensor(

(1, 4, 768)
torch.Size([828]) torch.Size([828, 384])
index tensor([0.6845, 0.6466, 0.6318, 0.6130, 0.5709], dtype=torch.float64) tensor([ 46, 468,  97, 105,  34])
The object of such satisfaction is called beautiful.
(1, 4, 768)
torch.Size([828]) torch.Size([828, 384])
index tensor([0.6845, 0.6466, 0.6318, 0.6130, 0.5709], dtype=torch.float64) tensor([ 46, 468,  97, 105,  34])
The object of such satisfaction is called beautiful.
(1, 4, 768)
torch.Size([828]) torch.Size([828, 384])
index tensor([0.6845, 0.6466, 0.6318, 0.6130, 0.5709], dtype=torch.float64) tensor([ 46, 468,  97, 105,  34])
The object of such satisfaction is called beautiful.
(1, 4, 768)
torch.Size([828]) torch.Size([828, 384])
index tensor([0.6845, 0.6466, 0.6318, 0.6130, 0.5709], dtype=torch.float64) tensor([ 46, 468,  97, 105,  34])
The object of such satisfaction is called beautiful.
(1, 4, 768)
torch.Size([828]) torch.Size([828, 384])
index tensor([0.6845, 0.6466, 0.6318, 0.6130, 0.5709], dtype=torch.float64) tensor(

In [18]:
%%timeit

# Best match SBERT
sbert_out = sbert_model.encode(test_sentence, convert_to_tensor=True)
cos_scores = util.pytorch_cos_sim(sbert_out, sbert_embeddings)[0]
score, idx = torch.topk(cos_scores, k=5)
print('index', score, idx)
print(CORPUS[idx[0]])

index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device

index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device

index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device

index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device

index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device

...

index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device

index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device

index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device

index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device

index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device='cuda:0')
A natural beauty is a beautiful thing; artificial beauty is a beautiful representation of a thing.
index tensor([0.6405, 0.6169, 0.5919, 0.5740, 0.5640], device='cuda:0') tensor([479,  46, 468,  66, 105], device

In [78]:
pipeline_out[:10], sbert_out.cpu().numpy()[:10]
# pipeline_out.shape, sbert_out.shape

(array([ 0.17699981,  0.55420619,  0.4372099 ,  0.29435544, -0.02624752,
        -0.48991437,  0.92081368, -0.44677985, -0.04004016,  0.33416032]),
 array([ 0.17700016,  0.5542062 ,  0.43721002,  0.29435566, -0.02624732,
        -0.48991424,  0.9208139 , -0.44678003, -0.04003995,  0.3341603 ],
       dtype=float32))

In [1]:
pd.options.display.max_colwidth = 500

test_sentence = 'Beautiful life'
# mat_vec = mat_vec.detach().numpy()

def extract_best_indices(m, topk, mask=None):
    """
    Use sum over all tokens
    m (array): cos matrix of shape (nb_in_tokens, nb_dict_tokens)
    topk (int): number of indices to return (from high to lowest in order)
    """
    # return the sum on all tokens of cosinus for each sentence
    if len(m.shape) > 1:
        cos_sim = np.mean(m, axis=0) 
    else: 
        cos_sim = m
    index = np.argsort(cos_sim)[::-1] # from highest idx to smallest score 
    if mask is not None:
        assert mask.shape == m.shape
        mask = mask[index]
    else:
        mask = np.ones(len(cos_sim))
    mask = np.logical_or(cos_sim[index] != 0, mask) #eliminate 0 cosine distance
    best_index = index[mask][:topk]  
    return best_index

def get_recommendations_bert(sentence, vec_mat):
    
    """
    Return the database sentences in order of highest cosine similarity relatively to each 
    token of the target sentence. 
    """
    if sentence.__class__ == 'str':
        sentence = [sentence] 
    input_vec = generate_embeddings(sentence, tokenizer, model)
    
    mat = cosine_similarity(input_vec, vec_mat)
#     print(input_vec.shape, mat.shape)
    # best cos sim for each token independantly
    best_index = extract_best_indices(mat, topk=3)
    return best_index

index = get_recommendations_bert(test_sentence, mat_vec)
philo_df[['sentence', 'author']].iloc[index]

NameError: name 'pd' is not defined

In [1]:
'test'.__class__

str

In [6]:
# list(philo_df.sentence.values)

In [11]:
# CLS start sentence
# SEP between two sentences for pair-training
# for example [CLS] [SEP]
CORPUS = list(philo_df.sentence.values[:500])
TGT = ['I love soup']
CORPUS = CORPUS + TGT

encodings = tokenizer(
    CORPUS, # the texts to be tokenized
    padding=True, # pad the texts to the maximum length (so that all outputs have the same length)
    return_tensors='pt' # return the tensors (not lists)
)
input_ids = encodings["input_ids"].tolist()

target  = tokenizer(
    TGT, # the texts to be tokenized
    padding=True, # pad the texts to the maximum length (so that all outputs have the same length)
    return_tensors='pt' # return the tensors (not lists)
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
encodings = encodings.to(device)

model = model.to(device)

cuda


In [9]:
for tokens in encodings['input_ids']:
    print(tokenizer.convert_ids_to_tokens(tokens))
    break


['[CLS]', ':', 'Perhaps', 'that', 'is', 'my', 'meaning', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


In [ ]:
with torch.no_grad():
    # get the model embeddings
    embeds = model(**encodings)
    
embeds = embeds[0] #contains embedding of tokens

# print(embeds.shape, target.shape)

In [3]:
from transformers import pipeline

model = pipeline("feature-extraction", model='bert-base-uncased') # model="distilgpt2"


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
vec_mat  = np.array(model(philo_df['sentence'].values.tolist()))# (nb_sentence, 5, 768)
vec_mat.shape

KeyboardInterrupt: 

In [32]:
from sklearn.metrics.pairwise import cosine_similarity

# using the last hidden state of the encoder (TO CHECK)
similarity = cosine_similarity(vec_mat[:,0,:])

In [56]:
pd.options.display.max_colwidth = 500

test_sentence = 'I am very angry'

def extract_best_indices(m, topk, mask=None):
    """
    Use sum over all tokens
    m (array): cos matrix of shape (nb_in_tokens, nb_dict_tokens)
    topk (int): number of indices to return (from high to lowest in order)
    """
    # return the sum on all tokens of cosinus for each sentence
    if len(m.shape) > 1:
        cos_sim = np.mean(m, axis=0) 
    else: 
        cos_sim = m
    index = np.argsort(cos_sim)[::-1] # from highest idx to smallest score 
    if mask is not None:
        assert mask.shape == m.shape
        mask = mask[index]
    else:
        mask = np.ones(len(cos_sim))
    mask = np.logical_or(cos_sim[index] != 0, mask) #eliminate 0 cosine distance
    best_index = index[mask][:topk]  
    return best_index

def get_recommendations_bert(sentence, dict_vec, model):
    
    """
    Return the database sentences in order of highest cosine similarity relatively to each 
    token of the target sentence. 
    """
    input_vec = np.array(model(sentence))[:,0,:]
    vec_mat = dict_vec[:,0,:]

#     input_vec = np.array(np.mean(model(sentence), axis=1))
#     vec_mat = np.mean(dict_vec, axis=1)
    
    mat = cosine_similarity(input_vec, vec_mat)
#     print(input_vec.shape, mat.shape)
    # best cos sim for each token independantly
    best_index = extract_best_indices(mat, topk=3)
    return best_index

index = get_recommendations_bert(test_sentence, vec_mat, model)
philo_df[['sentence', 'author']].iloc[index]

,sentence,author
107,What strange interlocutors have I found!,Nietzsche
2556,Why have I made this remark?,Plato
1764,Thank goodness I am not willing to let myself be torn to pieces!,Nietzsche


In [ ]:
# Sentence similarit with [CLS] token

In [ ]:
import torch.nn.functional as f # the sub-library containing different functions for manipulating with tensors


CLSs = embeds[:, 0, :]

print(CLSs.shape)
# normalize the CLS token embeddings
normalized = f.normalize(CLSs, p=2, dim=1)
normalized_tgt = f.normalize(CLSs, p=2, dim=1)
# calculate the cosine similarity
cls_dist = normalized.matmul(normalized_tgt.T)[-1,:]
cls_dist = cls_dist.new_ones(cls_dist.shape) - cls_dist
cls_dist = cls_dist.cpu().numpy()
cls_dist.shape


In [ ]:
idx = np.argmax(cls_dist)
print(idx)
CORPUS[idx]